In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import os
from torch.utils.data import Dataset, DataLoader
from SVarM_utils import *
from SVarM_models import Classifier
from scipy.io import loadmat

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
from torch.utils.data import Dataset, DataLoader
c=10

class MeshDataset(Dataset):
    def __init__(self, IDs):
        self.IDs = IDs

    def __len__(self):
        return len(self.IDs)

    def __getitem__(self, idx):
        file_path = self.IDs[idx]
        y = int(file_path.split("/")[1].split("_")[0])
        print(file_path)
        mesh = loadData(file_path.strip())  
        return mesh, torch.tensor([y])

matdata = loadmat("data_splits/MNIST_data_splits.mat")
test_data = matdata['test_data'].tolist()
train_data = matdata['train_data'].tolist()
val_dataset = MeshDataset(test_data)
train_dataset = MeshDataset(train_data)

In [3]:
model = Classifier(10)
sum(p.numel() for p in model.parameters() if p.requires_grad)

1850

In [4]:
batch_size =1
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, collate_fn=lambda x: list(zip(*x)))
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, collate_fn=lambda x: list(zip(*x)))
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=.05)
epochs = 20

In [5]:
for epoch in range(epochs):
        model.train()
        count = 0
        train_loss = 0.0
        for meshs, y in train_loader: 
            y = torch.stack(y)
            optimizer.zero_grad()
            outputs = model(meshs)
            loss = criterion(outputs, y)
            loss.backward()
            optimizer.step()

            train_loss += loss.item()
            count=count+1
            
            print(f"Epoch [{count}/{len(train_loader)}] - Train Loss: {loss.item():.4f}", end ="\r")

        train_loss /= len(train_loader)

        # Validation phase
        model.eval()
        val_loss = 0.0
        correct = 0
        total = 0

        with torch.no_grad():
            for meshs, y in val_loader:
                y = torch.stack(y)
                outputs = model(meshs)
                loss = criterion(outputs, y)
                val_loss += loss.item()
                # Compute accuracy
                _, predicted = torch.max(outputs, 1)  # Get predicted class (index of max logit)
                total += y.size(0)  # Total number of samples
                correct += (predicted == y).sum().item()  # Count correct predictions
        accuracy = 100 * correct / total 

        val_loss /= len(val_loader)

        print(f"Epoch [{epoch+1}/{epochs}] - Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}, Accuracy: {accuracy:.2f}%")

MNIST_remeshed/6_39395.ply
[Open3D WARNING] Read PLY failed: unable to open file: MNIST_remeshed\6_39395.ply


RuntimeError: Expected target size [1, 10], got [1, 1]